# YOLO

## Object localization

- target label $y = 
\begin{bmatrix}
    p_{c} \\ 
    b_{x} \\ 
    b_{y} \\
    b_{h} \\
    b_{w} \\
    c_{1} \\
    c_{2} \\
    c_{3} \\
\end{bmatrix}$
 
- $p_{c}$: is there an object?
- $b_{x}$: coordinate
- $b_{y}$: coordinate
- $b_{h}$: height
- $b_{w}$: width
- $c_{1}$: class #1
- $c_{2}$: class #2
- $c_{3}$: class #3
- $L(\hat{y}, y) = (\hat{y}_{1} - y_{1})^{2} + (\hat{y}_{2} - y_{2})^{2} + \dots + (\hat{y}_{8} - y_{8})^{2}$ if $y_{1} = 1$ 
- $L(\hat{y}, y) = (\hat{y}_{1} - y_{1})^{2}$ if $y_{1} = 0$

## Sliding windows

- create boxes and move it around inside the image. when image recognition algorithm detects the object, return. when finished, change the size of boxes and move them around again

## Convolution implementation of sliding windows

- 14x14x3 $\xrightarrow{5x5}$ 10x10x16 $\xrightarrow{2x2, POOL}$ 5x5x16 $\xrightarrow{5x5, FC}$ 1x1x400 $\xrightarrow{1x1, FC}$ 1x1x400 $\xrightarrow{1x1, FC}$ 1x1x4
- 16x16x3 $\xrightarrow{5x5}$ 12x12x16 $\xrightarrow{2x2, POOL}$ 6x6x16 $\xrightarrow{5x5, FC}$ 2x2x400 $\xrightarrow{1x1, FC}$ 2x2x400 $\xrightarrow{1x1, FC}$ 2x2x4
- 28x28x3 $\xrightarrow{5x5}$ 24x24x16 $\xrightarrow{2x2, POOL}$ 12x12x16 $\xrightarrow{5x5, FC}$ 8x8x400 $\xrightarrow{1x1, FC}$ 8x8x400 $\xrightarrow{1x1, FC}$ 8x8x4

## Bounding box predictions

- split the image into multiple grid cells
- assign object to grid cell by the midpoint of the object

## Intersection over union

- measure of the overlap between two bounding boxes
- size of intersection / size of predicted bounding box
- correct if IoU $\ge 0.5$

## Non-max suppression

- cleans up multiple detections of same object
- discard all boxes with $p_{c} \le 0.6$
- while there are any remaining boxes
    - pick the box with the largest $p_{c}$, output that as a prediction
    - discard any remaining box with IoU $\ge 0.5$ with the box output in the previous step
    
## Anchor boxes

- overlapping objects? create multiple (anchor) boxes
- target label $y = 
\begin{bmatrix}
    p_{c} \\ 
    b_{x} \\ 
    b_{y} \\
    b_{h} \\
    b_{w} \\
    c_{1} \\
    c_{2} \\
    c_{3} \\
    p_{c} \\ 
    b_{x} \\ 
    b_{y} \\
    b_{h} \\
    b_{w} \\
    c_{1} \\
    c_{2} \\
    c_{3} \\
\end{bmatrix}$
- each object in training image is assigned to grid cell that contains object's midpoint and anchor box for the grid cell with highest IoU

## YOLO algorithm

- ex. training: 3x3x2 (number of anchors) x8 (5 + number of classes)
- output: 3x3x16


## Example

<center>
<video width="400" height="200" src="img/road_video_compressed2.mp4" type="video/mp4" controls>
</video>
</center>

- Gather images from the video.
- Draw box around every car found.

<img src="img/box_label.png" style="width:500px;height:250;">

### Packages

In [3]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import random
import colorsys
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model

%matplotlib inline

### Yolo

- Only one forward prop to make prediction.

#### Inputs 

#### Outputs

#### Anchor boxes

#### Encoding

#### Class score


In [2]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):
    """Filters YOLO boxes by thresholding on object and class confidence.
    
    Arguments:
    box_confidence -- tensor of shape (19, 19, 5, 1)
    boxes -- tensor of shape (19, 19, 5, 4)
    box_class_probs -- tensor of shape (19, 19, 5, 80)
    threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    
    Returns:
    scores -- tensor of shape (None,), containing the class probability score for selected boxes
    boxes -- tensor of shape (None, 4), containing (b_x, b_y, b_h, b_w) coordinates of selected boxes
    classes -- tensor of shape (None,), containing the index of the class detected by the selected boxes
    
    Note: "None" is here because you don't know the exact number of selected boxes, as it depends on the threshold. 
    For example, the actual output size of scores would be (10,) if there are 10 boxes.
    """
    
    # Step 1: Compute box scores
    box_scores = box_confidence * box_class_probs
    
    # Step 2: Find the box_classes using the max box_scores, keep track of the corresponding score
    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1, keepdims=False)
    
    # Step 3: Create a filtering mask based on "box_class_scores" by using "threshold". The mask should have the
    # same dimension as box_class_scores, and be True for the boxes you want to keep (with probability >= threshold)
    filtering_mask = box_class_scores >= threshold
    
    # Step 4: Apply the mask to box_class_scores, boxes and box_classes
    scores = tf.boolean_mask(box_class_scores, filtering_mask)
    boxes = tf.boolean_mask(boxes, filtering_mask)
    classes = tf.boolean_mask(box_classes, filtering_mask)
    
    return scores, boxes, classes

### Non-max supression

In [4]:
def iou(box1, box2):
    """Implement the intersection over union (IoU) between box1 and box2
    
    Arguments:
    box1 -- first box, list object with coordinates (box1_x1, box1_y1, box1_x2, box_1_y2)
    box2 -- second box, list object with coordinates (box2_x1, box2_y1, box2_x2, box2_y2)
    """

    # Assign variable names to coordinates for clarity
    (box1_x1, box1_y1, box1_x2, box1_y2) = box1
    (box2_x1, box2_y1, box2_x2, box2_y2) = box2
    
    # Calculate the (yi1, xi1, yi2, xi2) coordinates of the intersection of box1 and box2. Calculate its Area.
    xi1 = max(box1_x1, box2_x1)
    yi1 = max(box1_y1, box2_y1)
    xi2 = min(box1_x2, box2_x2)
    yi2 = min(box1_y2, box2_y2)
    inter_width = xi2 - xi1
    inter_height = yi2 - yi1
    inter_area = max(inter_width, 0) * max(inter_height, 0)

    # Calculate the Union area by using Formula: Union(A,B) = A + B - Inter(A,B)
    box1_area = (box1_x2 - box1_x1) * (box1_y2 - box1_y1)
    box2_area = (box2_x2 - box2_x1) * (box2_y2 - box2_y1)
    union_area = box1_area + box2_area - inter_area
    
    # compute the IoU
    iou = inter_area / union_area
    
    return iou

In [5]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    """
    Applies Non-max suppression (NMS) to set of boxes
    
    Arguments:
    scores -- tensor of shape (None,), output of yolo_filter_boxes()
    boxes -- tensor of shape (None, 4), output of yolo_filter_boxes() that have been scaled to the image size (see later)
    classes -- tensor of shape (None,), output of yolo_filter_boxes()
    max_boxes -- integer, maximum number of predicted boxes you'd like
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (, None), predicted score for each box
    boxes -- tensor of shape (4, None), predicted box coordinates
    classes -- tensor of shape (, None), predicted class for each box
    
    Note: The "None" dimension of the output tensors has obviously to be less than max_boxes. Note also that this
    function will transpose the shapes of scores, boxes, classes. This is made for convenience.
    """
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')     # tensor to be used in tf.image.non_max_suppression()
    K.get_session().run(tf.variables_initializer([max_boxes_tensor])) # initialize variable max_boxes_tensor
    
    # Use tf.image.non_max_suppression() to get the list of indices corresponding to boxes you keep
    nms_indices = tf.image.non_max_suppression(boxes, scores, max_boxes_tensor, iou_threshold=iou_threshold, name=None)
    
    # Use K.gather() to select only nms_indices from scores, boxes and classes
    scores = K.gather(scores, nms_indices)
    boxes = K.gather(boxes, nms_indices)
    classes = K.gather(classes, nms_indices)
    
    return scores, boxes, classes

In [6]:
def scale_boxes(boxes, image_shape):
    """ Scales the predicted boxes in order to be drawable on the image"""
    height = image_shape[0]
    width = image_shape[1]
    image_dims = K.stack([height, width, height, width])
    image_dims = K.reshape(image_dims, [1, 4])
    boxes = boxes * image_dims
    return boxes

In [7]:
def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    """
    Converts the output of YOLO encoding (a lot of boxes) to your predicted boxes along with their scores, box coordinates and classes.
    
    Arguments:
    yolo_outputs -- output of the encoding model (for image_shape of (608, 608, 3)), contains 4 tensors:
                    box_confidence: tensor of shape (None, 19, 19, 5, 1)
                    box_xy: tensor of shape (None, 19, 19, 5, 2)
                    box_wh: tensor of shape (None, 19, 19, 5, 2)
                    box_class_probs: tensor of shape (None, 19, 19, 5, 80)
    image_shape -- tensor of shape (2,) containing the input shape, in this notebook we use (608., 608.) (has to be float32 dtype)
    max_boxes -- integer, maximum number of predicted boxes you'd like
    score_threshold -- real value, if [ highest class probability score < threshold], then get rid of the corresponding box
    iou_threshold -- real value, "intersection over union" threshold used for NMS filtering
    
    Returns:
    scores -- tensor of shape (None, ), predicted score for each box
    boxes -- tensor of shape (None, 4), predicted box coordinates
    classes -- tensor of shape (None,), predicted class for each box
    """
    
    # Retrieve outputs of the YOLO model (≈1 line)
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs

    # Convert boxes to be ready for filtering functions (convert boxes box_xy and box_wh to corner coordinates)
    boxes = yolo_boxes_to_corners(box_xy, box_wh)

    # Use one of the functions you've implemented to perform Score-filtering with a threshold of score_threshold (≈1 line)
    scores, boxes, classes = yolo_filter_boxes(box_confidence, boxes, box_class_probs, score_threshold)
    
    # Scale boxes back to original image shape.
    boxes = scale_boxes(boxes, image_shape)

    # Use one of the functions you've implemented to perform Non-max suppression with 
    # maximum number of boxes set to max_boxes and a threshold of iou_threshold (≈1 line)
    scores, boxes, classes = yolo_non_max_suppression(scores, boxes, classes, max_boxes, iou_threshold)
    
    return scores, boxes, classes